In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from pandas.tseries.offsets import DateOffset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime
import time
import shap
from sklearn.model_selection import GridSearchCV, ShuffleSplit, cross_validate, train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, confusion_matrix, classification_report
import xgboost as xgb
from pandas_profiling import ProfileReport

pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

In [43]:
df = pd.read_csv("data/nba.csv")

In [44]:
def feature_engineering(df):
    # adding wins above replacement for year and career
    # do we want to aggregate this via a sum or pieced average
    df = df.loc[~(df.Tm == "TOT")].copy()
    df["birth_date"] = pd.to_datetime(df.birth_date, infer_datetime_format = True)
    df["WOR"] = df.VORP * 2.7
    df["WOR_Year"] = df.groupby(["name", "Year"]).WOR.transform("sum")
    df["WOR_Career_avg"] = df.groupby("name").WOR_Year.transform("mean")
    df["WOR_Career_tot"] = df.groupby("name").WOR_Year.transform("sum")
    # HOF identifier
    hof = list(set(df.loc[df.Player.str.contains("\*"),"Player"].str.replace("*","").values))
    df.loc[df.name.isin(hof),"HOF"] = 1
    df.loc[df.HOF.isnull(), "HOF"] = 0
    
    return df

In [45]:
df = feature_engineering(df)

In [32]:
df.describe()

,Year,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,year_start,year_end,weight,HOF,WOR,WOR_Year,WOR_Career_avg,WOR_Career_tot,total_games_year
count,23099.000000,23092.000000,23099.000000,17130.000000,22679.000000,22669.000000,23011.000000,17694.000000,22997.000000,19547.000000,19547.000000,20285.000000,21216.000000,19547.000000,19547.000000,18378.000000,18437.000000,23094.000000,23094.000000,23094.000000,22669.000000,19552.000000,19552.000000,19552.000000,19552.000000,23099.000000,23099.000000,22997.000000,17783.000000,17783.000000,14479.000000,23099.000000,23099.000000,22964.000000,22997.000000,23099.000000,23099.000000,22248.00000,19552.000000,19552.000000,22828.000000,23099.000000,19552.000000,19552.000000,18442.000000,23099.000000,23099.000000,23099.000000,23099.000000,23094.000000,23099.000000,19552.000000,23099.000000,23099.000000,23099.000000,23099.000000
mean,1992.516819,26.657500,50.769817,24.322358,1225.133736,12.567771,0.494202,0.157633,0.326596,6.193759,13.717629,9.959487,13.059309,1.649900,1.429068,15.054930,18.962836,1.317394,1.257435,2.575725,0.066457,-1.714975,-0.513932,-2.228652,0.609232,199.226633,437.448807,0.432133,22.347410,63.913794,0.248568,182.022252,388.244123,0.446732,0.451855,104.331270,139.118706,0.72017,63.293065,149.757774,227.898020,116.905407,40.611702,25.307999,75.143206,117.262782,519.988917,1988.374951,1996.718646,212.231142,0.066453,1.644926,1.556506,1.556506,30.004353,70.284168
std,17.266782,3.876052,26.843409,29.133838,954.175538,6.169216,0.096095,0.187946,0.225661,4.917502,6.701538,5.081645,9.271058,1.032339,1.822649,7.014237,5.553593,2.183070,1.300184,3.129046,0.104859,3.865488,2.305991,4.785082,1.372243,192.074484,405.015142,0.097761,38.969975,103.428891,0.178031,183.311600,378.202903,0.101507,0.100965,115.370684,148.487788,0.14203,68.254409,148.403652,231.518011,138.294491,39.468076,38.364513,68.787981,85.756856,502.903836,17.070641,17.697043,26.493387,0.249078,3.705055,3.958563,3.028616,78.814538,45.890567
min,1950.000000,18.000000,1.000000,0.000000,0.000000,-90.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.100000,-1.000000,-2.800000,-2.519000,-73.800000,-30.400000,-86.700000,-2.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1947.000000,1947.000000,133.000000,0.000000,-7.020000,-7.560000,-3.780000,-52.920000,1.000000
25%,1981.000000,24.000000,27.000000,0.000000,335.000000,9.900000,0.460000,0.005000,0.208000,2.600000,8.800000,5.900000,6.500000,1.100000,0.300000,11.300000,15.400000,0.000000,0.200000,0.200000,0.032000,-3.300000,-1.700000,-4.100000,-0.100000,40.000000,97.000000,0.394000,0.000000,1.000000,0.095000,34.000000,81.000000,0.409000,0.416000,18.000000,26.500000,0.65900,12.000000,33.000000,51.000000,19.000000,9.000000,3.000000,18.000000,39.000000,105.000000,1977.000000,1985.000000,190.000000,0.000000,-0.270000,-0.270000,-0.180000,-0.810000,45.000000
50%,1995.000000,26.000000,58.000000,8.000000,1075.000000,12.800000,0.507000,0.062000,0.296000,5.400000,12.800000,9.200000,10.600000,1.500000,0.900000,14.100000,18.700000,0.500000,0.900000,1.500000,0.077000,-1.400000,-0.500000,-1.700000,0.000000,143.000000,326.000000,0.440000,2.000000,11.000000,0.290000,125.000000,275.000000,0.458000,0.465000,65.000000,90.000000,0.74500,39.000000,109.000000,163.000000,69.000000,30.000000,12.000000,56.000000,110.000000,373.000000,1990.000000,2000.000000,210.000000,0.000000,0.000000,0.000000,0.320625,2.700000,71.000000
75%,2007.000000,29.000000,76.000000,48.000000,2008.000000,15.700000,0.545000,0.286000,0.400000,9.000000,18.100000,13.500000,17.500000,2.100000,1.900000,17.600000,22.200000,2.100000,1.900000,4.000000,0.117000,0.400000,0.800000,0.50000

In [38]:
df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,name,year_start,year_end,position,height,weight,birth_date,college,HOF,WOR,WOR_Year,WOR_Career_avg,WOR_Career_tot,total_games_year
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,NaN,0.467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,3.6,3.5,NaN,NaN,NaN,NaN,NaN,144.0,516.0,0.279,NaN,NaN,NaN,144.0,516.0,0.279,0.279,170.0,241.0,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0,Curly Armstrong,1949,1951,G-F,5-11,170.0,"November 1, 1918",Indiana University,0.0,NaN,0.0,0.0,0.0,63.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,NaN,0.387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6,0.6,2.2,NaN,NaN,NaN,NaN,NaN,102.0,274.0,0.372,NaN,NaN,NaN,102.0,274.0,0.372,0.372,75.0,106.0,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0,Cliff Barker,1950,1952,G,6-2,185.0,"January 15, 1921",University of Kentucky,0.0,NaN,0.0,0.0,0.0,49.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,NaN,0.259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9,2.8,3.6,NaN,NaN,NaN,NaN,NaN,174.0,499.0,0.349,NaN,NaN,NaN,174.0,499.0,0.349,0.349,90.0,129.0,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0,Leo Barnhorst,1950,1954,F-G,6-4,190.0,"May 11, 1924",University of Notre Dame,0.0,NaN,0.0,0.0,0.0,67.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,NaN,0.378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,-0.1,-0.6,NaN,NaN,NaN,NaN,NaN,21.0,82.0,0.256,NaN,NaN,NaN,21.0,82.0,0.256,0.256,17.0,31.0,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0,Ed Bartels,1950,1951,F,6-5,195.0,"October 8, 1925",North Carolina State University,0.0,NaN,0.0,0.0,0.0,15.0
5,1950.0,Ed Bartels,F,24.0,NYK,2.0,NaN,NaN,NaN,0.376,NaN,0.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,4.0,0.250,NaN,NaN,NaN,1.0,4.0,0.250,0.250,2.0,3.0,0.667,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2.0,4.0,Ed Bartels,1950,1951,F,6-5,195.0,"October 8, 1925",North Carolina State University,0.0,NaN,0.0,0.0,0.0,15.0


In [97]:
df["total_games_year"] = df.groupby(["name","Year"]).G.transform("sum")
df["age_year_diff"] = abs((df.Age - (df.Year - df.birth_date.dt.year)))
df["bday_count"] = df.groupby(["name"]).birth_date.transform("nunique")

In [37]:
double_names = df.loc[df.total_games_year > 83,"name"].unique()
double_names

array(['Jack Turner', 'Jim Paxson', 'Bob Duffy', 'Luke Jackson',
       'Matt Guokas', 'Bill Bradley', 'John Tresvant', 'Walt Bellamy',
       'Fred Hetzel', 'Howard Komives', 'Greg Smith', 'Al Tucker',
       'Sam Williams', 'Mike Davis', 'McCoy McLemore', 'Jim Washington',
       'George Johnson', 'Dan Anderson', 'Don Smith', 'Nate Williams',
       'Bobby Wilson', 'Gar Heard', 'Roger Brown', 'Tom Henderson',
       'Bobby Jones', 'Eddie Johnson', 'Mike Dunleavy',
       'Gerald Henderson', 'Billy Paultz', 'Larry Drew', 'Billy McKinney',
       'Dave Robisch', 'Mike Mitchell', 'Larry Nance', 'Walker Russell',
       'Charles Jones', 'Patrick Ewing', 'Ken Johnson', 'Tyrone Corbin',
       'Charles Smith', 'Reggie Williams', 'Tim Hardaway', 'Glen Rice',
       'Michael Smith', 'Dee Brown', 'Gary Payton', 'Thurl Bailey',
       'Larry Johnson', 'Chris Smith', 'Glenn Robinson', 'Mark Davis',
       'Cedric Henderson', 'Chris Gatling', 'Mike James',
       'Shareef Abdur-Rahim', 'Theo Rat

In [96]:
print(len(df.loc[df.total_games_year > 83]))
print(len(df.loc[df.bday_count > 1]))
print(len(df.loc[((df.total_games_year > 83) | (df.bday_count > 1)) ]))
print(len(df.loc[((df.total_games_year > 83) & (df.bday_count > 1)) ]))

909
1121
1165
865


In [48]:
 df.loc[df.total_games_year > 83, ["name","birth_date","Year","Age"]]

,name,birth_date,Year,Age
1011,Jack Turner,1930-06-29,1955.0,24.0
1012,Jack Turner,1939-06-05,1955.0,24.0
1198,Jim Paxson,1932-12-19,1957.0,24.0
1199,Jim Paxson,1957-07-09,1957.0,24.0
1309,Jim Paxson,1932-12-19,1958.0,25.0
...,...,...,...,...
24993,David Lee,1983-04-29,2017.0,33.0
25064,Larry Nance,1959-02-12,2017.0,24.0
25065,Larry Nance,1993-01-01,2017.0,24.0
25141,Glenn Robinson,1973-01-10,2017.0,23.0


In [106]:
(df.age_year_diff > 1).sum()

596

In [108]:
df.loc[df.age_year_diff > 1].sort_values(by = ["Player","Year"]).head(50)

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,name,year_start,year_end,position,height,weight,birth_date,college,WOR,WOR_Year,WOR_Career_avg,WOR_Career_tot,HOF,total_games_year,age_year_diff,bday_count
2499,1968.0,Bill Bradley*,SF,24.0,NYK,45.0,NaN,874.0,11.7,0.465,NaN,0.305,NaN,NaN,NaN,20.5,NaN,NaN,NaN,NaN,0.2,0.8,1.1,0.058,NaN,NaN,NaN,NaN,142.0,341.0,0.416,NaN,NaN,NaN,142.0,341.0,0.416,0.416,76.0,104.0,0.731,NaN,NaN,113.0,137.0,NaN,NaN,NaN,138.0,360.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,NaN,0.00,-0.432000,-8.64,1.0,90.0,3.0,2
2687,1969.0,Bill Bradley*,SF,25.0,NYK,82.0,NaN,2413.0,13.1,0.481,NaN,0.267,NaN,NaN,NaN,16.9,NaN,NaN,NaN,NaN,1.9,2.7,4.7,0.093,NaN,NaN,NaN,NaN,407.0,948.0,0.429,NaN,NaN,NaN,407.0,948.0,0.429,0.429,206.0,253.0,0.814,NaN,NaN,350.0,302.0,NaN,NaN,NaN,295.0,1020.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,NaN,0.00,-0.432000,-8.64,1.0,164.0,3.0,2
2894,1970.0,Bill Bradley*,SF,26.0,NYK,67.0,NaN,2098.0,13.7,0.498,NaN,0.196,NaN,NaN,NaN,16.7,NaN,NaN,NaN,NaN,1.6,3.3,4.9,0.112,NaN,NaN,NaN,NaN,413.0,897.0,0.460,NaN,NaN,NaN,413.0,897.0,0.460,0.460,145.0,176.0,0.824,NaN,NaN,239.0,268.0,NaN,NaN,NaN,219.0,971.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,NaN,0.00,-0.432000,-8.64,1.0,134.0,3.0,2
3107,1971.0,Bill Bradley*,SF,27.0,NYK,78.0,NaN,2300.0,13.3,0.490,NaN,0.192,NaN,NaN,5.2,16.5,NaN,NaN,NaN,NaN,1.8,3.6,5.4,0.114,NaN,NaN,NaN,NaN,413.0,912.0,0.453,NaN,NaN,NaN,413.0,912.0,0.453,0.453,144.0,175.0,0.823,NaN,NaN,260.0,280.0,NaN,NaN,NaN,245.0,970.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,NaN,0.00,-0.432000,-8.64,1.0,156.0,3.0,2
3353,1972.0,Bill Bradley*,SF,28.0,NYK,78.0,NaN,2780.0,12.9,0.502,NaN,0.183,NaN,NaN,4.4,16.0,NaN,NaN,NaN,NaN,2.7,2.8,5.4,0.094,NaN,NaN,NaN,NaN,504.0,1085.0,0.465,NaN,NaN,NaN,504.0,1085.0,0.465,0.465,169.0,199.0,0.849,NaN,NaN,250.0,315.0,NaN,NaN,NaN,254.0,1177.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,NaN,0.00,-0.432000,-8.64,1.0,156.0,3.0,2
3630,1973.0,Bill Bradley*,SF,29.0,NYK,82.0,NaN,2998.0,13.8,0.493,NaN,0.155,NaN,NaN,5.0,16.8,NaN,NaN,NaN,NaN,2.9,4.1,7.0,0.112,NaN,NaN,NaN,NaN,575.0,1252.0,0.459,NaN,NaN,NaN,575.0,1252.0,0.459,0.459,169.0,194.0,0.871,NaN,NaN,301.0,367.0,NaN,NaN,NaN,273.0,1319.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,NaN,0.00,-0.432000,-8.64,1.0,164.0,3.0,2
3907,1974.0,Bill Bradley*,SF,30.0,NYK,82.0,NaN,2813.0,11.1,0.485,NaN,0.150,2.2,7.2,4.7,12.3,0.7,0.4,NaN,NaN,1.6,2.6,4.2,0.071,-1.5,-0.7,-2.2,-0.2,502.0,1112.0,0.451,NaN,NaN,NaN,502.0,1112.0,0.451,0.451,146.0,167.0,0.874,59.0,194.0,253.0,242.0,42.0,21.0,NaN,278.0,1150.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,-0.54,-1.08,-0.432000,-8.64,1.0,164.0,3.0,2
4179,1975.0,Bill Bradley*,SF,31.0,NYK,79.0,NaN,2787.0,11.1,0.473,NaN,0.159,2.4,7.1,4.7,12.9,1.2,0.3,NaN,NaN,1.4,1.4,2.8,0.048,-1.3,-0.8,-2.2,-0.1,452.0,1036.0,0.436,NaN,NaN,NaN,452.0,1036.0,0.436,0.436,144.0,165.0,0.873,65.0,186.0,251.0,247.0,74.0,18.0,NaN,283.0,1048.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,-0.27,-0.54,-0.432000,-8.64,1.0,158.0,3.0,2
4458,1976.0,Bill Bradley*,SF,32.0,NYK,82.0,NaN,2709.0,10.2,0.471,NaN,0.163,1.8,7.2,4.4,12.6,1.2,0.4,NaN,NaN,1.0,1.5,2.5,0.044,-1.8,-0.3,-2.1,-0.1,392.0,906.0,0.433,NaN,NaN,NaN,392.0,906.0,0.433,0.433,130.0,148.0,0.878,47.0,187.0,234.0,247.0,68.0,18.0,NaN,256.0,914.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,-0.27,-0.54,-0.432000,-8.64,1.0,164.0,3.0,2
4741,1977.0,Bill Bradley*,SF,33.0,NYK,67.0,NaN,1027.0,9.5,0.492,NaN,0.153,2.8,7.6,5.3,15.5,1.1,0.4,NaN,NaN,0.3,0.5,0.8,0.038,-2.8,-0.8,-3.6,-0.4,127.0,274.0,0.464,NaN,NaN,NaN,127.0,274.0,0.464,0.464,34.0,42.0,0.810,27.0,76.0,103.0,128.0,25.0,8.0,NaN,122.0,288.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,-1.08,-2.16,-0.432000,-8.64,1.0,134.0,3.0,2


In [91]:
df.loc[df.Player == "Freddie Lewis"]

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,name,year_start,year_end,position,height,weight,birth_date,college,WOR,WOR_Year,WOR_Career_avg,WOR_Career_tot,HOF,total_games_year,age_year_diff,bday_count
142,1950.0,Freddie Lewis,F-G,29.0,BLB,18.0,NaN,NaN,NaN,0.266,NaN,0.173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.1,0.2,-0.9,NaN,NaN,NaN,NaN,NaN,25.0,110.0,0.227,NaN,NaN,NaN,25.0,110.0,0.227,0.227,13.0,19.0,0.684,NaN,NaN,NaN,18.0,NaN,NaN,NaN,23.0,63.0,Freddie Lewis,1949,1950,F-G,6-2,195.0,1921-01-06,Eastern Kentucky University,NaN,0.00,-0.54,-4.32,0.0,68.0,0.0,2
143,1950.0,Freddie Lewis,F-G,29.0,BLB,18.0,NaN,NaN,NaN,0.266,NaN,0.173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.1,0.2,-0.9,NaN,NaN,NaN,NaN,NaN,25.0,110.0,0.227,NaN,NaN,NaN,25.0,110.0,0.227,0.227,13.0,19.0,0.684,NaN,NaN,NaN,18.0,NaN,NaN,NaN,23.0,63.0,Freddie Lewis,1967,1977,G,6-0,175.0,1943-07-01,Arizona State University,NaN,0.00,-0.54,-4.32,0.0,68.0,22.0,2
144,1950.0,Freddie Lewis,F-G,29.0,PHW,16.0,NaN,NaN,NaN,0.339,NaN,0.176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.3,0.3,0.0,NaN,NaN,NaN,NaN,NaN,21.0,74.0,0.284,NaN,NaN,NaN,21.0,74.0,0.284,0.284,12.0,13.0,0.923,NaN,NaN,NaN,7.0,NaN,NaN,NaN,17.0,54.0,Freddie Lewis,1949,1950,F-G,6-2,195.0,1921-01-06,Eastern Kentucky University,NaN,0.00,-0.54,-4.32,0.0,68.0,0.0,2
145,1950.0,Freddie Lewis,F-G,29.0,PHW,16.0,NaN,NaN,NaN,0.339,NaN,0.176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.3,0.3,0.0,NaN,NaN,NaN,NaN,NaN,21.0,74.0,0.284,NaN,NaN,NaN,21.0,74.0,0.284,0.284,12.0,13.0,0.923,NaN,NaN,NaN,7.0,NaN,NaN,NaN,17.0,54.0,Freddie Lewis,1967,1977,G,6-0,175.0,1943-07-01,Arizona State University,NaN,0.00,-0.54,-4.32,0.0,68.0,22.0,2
2423,1967.0,Freddie Lewis,PG,23.0,CIN,32.0,NaN,334.0,10.5,0.436,NaN,0.268,NaN,NaN,NaN,15.8,NaN,NaN,NaN,NaN,-0.2,0.2,0.0,0.004,NaN,NaN,NaN,NaN,60.0,153.0,0.392,NaN,NaN,NaN,60.0,153.0,0.392,0.392,29.0,41.0,0.707,NaN,NaN,44.0,40.0,NaN,NaN,NaN,49.0,149.0,Freddie Lewis,1949,1950,F-G,6-2,195.0,1921-01-06,Eastern Kentucky University,NaN,0.00,-0.54,-4.32,0.0,64.0,23.0,2
2424,1967.0,Freddie Lewis,PG,23.0,CIN,32.0,NaN,334.0,10.5,0.436,NaN,0.268,NaN,NaN,NaN,15.8,NaN,NaN,NaN,NaN,-0.2,0.2,0.0,0.004,NaN,NaN,NaN,NaN,60.0,153.0,0.392,NaN,NaN,NaN,60.0,153.0,0.392,0.392,29.0,41.0,0.707,NaN,NaN,44.0,40.0,NaN,NaN,NaN,49.0,149.0,Freddie Lewis,1967,1977,G,6-0,175.0,1943-07-01,Arizona State University,NaN,0.00,-0.54,-4.32,0.0,64.0,1.0,2
4908,1977.0,Freddie Lewis,PG,33.0,IND,32.0,NaN,552.0,11.3,0.481,NaN,0.387,2.9,5.4,4.1,13.6,1.5,0.2,NaN,NaN,0.4,0.2,0.5,0.047,-2.3,-2.6,-5.0,-0.4,81.0,199.0,0.407,NaN,NaN,NaN,81.0,199.0,0.407,0.407,62.0,77.0,0.805,17.0,30.0,47.0,56.0,18.0,2.0,NaN,58.0,224.0,Freddie Lewis,1949,1950,F-G,6-2,195.0,1921-01-06,Eastern Kentucky University,-1.08,-2.16,-0.54,-4.32,0.0,64.0,23.0,2
4909,1977.0,Freddie Lewis,PG,33.0,IND,32.0,NaN,552.0,11.3,0.481,NaN,0.387,2.9,5.4,4.1,13.6,1.5,0.2,NaN,NaN,0.4,0.2,0.5,0.047,-2.3,-2.6,-5.0,-0.4,81.0,199.0,0.407,NaN,NaN,NaN,81.0,199.0,0.407,0.407,62.0,77.0,0.805,17.0,30.0,47.0,56.0,18.0,2.0,NaN,58.0,224.0,Freddie Lewis,1967,1977,G,6-0,175.0,1943-07-01,Arizona State University,-1.08,-2.16,-0.54,-4.32,0.0,64.0,1.0,2


In [100]:
df.loc[df.name == "Bill Bradley"]

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,name,year_start,year_end,position,height,weight,birth_date,college,WOR,WOR_Year,WOR_Career_avg,WOR_Career_tot,HOF,total_games_year,age_year_diff,bday_count
2499,1968.0,Bill Bradley*,SF,24.0,NYK,45.0,NaN,874.0,11.7,0.465,NaN,0.305,NaN,NaN,NaN,20.5,NaN,NaN,NaN,NaN,0.2,0.8,1.1,0.058,NaN,NaN,NaN,NaN,142.0,341.0,0.416,NaN,NaN,NaN,142.0,341.0,0.416,0.416,76.0,104.0,0.731,NaN,NaN,113.0,137.0,NaN,NaN,NaN,138.0,360.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,NaN,0.00,-0.432,-8.64,1.0,90.0,3.0,2
2500,1968.0,Bill Bradley*,SF,24.0,NYK,45.0,NaN,874.0,11.7,0.465,NaN,0.305,NaN,NaN,NaN,20.5,NaN,NaN,NaN,NaN,0.2,0.8,1.1,0.058,NaN,NaN,NaN,NaN,142.0,341.0,0.416,NaN,NaN,NaN,142.0,341.0,0.416,0.416,76.0,104.0,0.731,NaN,NaN,113.0,137.0,NaN,NaN,NaN,138.0,360.0,Bill Bradley,1968,1977,F-G,6-5,205.0,1943-07-28,Princeton University,NaN,0.00,-0.432,-8.64,1.0,90.0,1.0,2
2687,1969.0,Bill Bradley*,SF,25.0,NYK,82.0,NaN,2413.0,13.1,0.481,NaN,0.267,NaN,NaN,NaN,16.9,NaN,NaN,NaN,NaN,1.9,2.7,4.7,0.093,NaN,NaN,NaN,NaN,407.0,948.0,0.429,NaN,NaN,NaN,407.0,948.0,0.429,0.429,206.0,253.0,0.814,NaN,NaN,350.0,302.0,NaN,NaN,NaN,295.0,1020.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,NaN,0.00,-0.432,-8.64,1.0,164.0,3.0,2
2688,1969.0,Bill Bradley*,SF,25.0,NYK,82.0,NaN,2413.0,13.1,0.481,NaN,0.267,NaN,NaN,NaN,16.9,NaN,NaN,NaN,NaN,1.9,2.7,4.7,0.093,NaN,NaN,NaN,NaN,407.0,948.0,0.429,NaN,NaN,NaN,407.0,948.0,0.429,0.429,206.0,253.0,0.814,NaN,NaN,350.0,302.0,NaN,NaN,NaN,295.0,1020.0,Bill Bradley,1968,1977,F-G,6-5,205.0,1943-07-28,Princeton University,NaN,0.00,-0.432,-8.64,1.0,164.0,1.0,2
2894,1970.0,Bill Bradley*,SF,26.0,NYK,67.0,NaN,2098.0,13.7,0.498,NaN,0.196,NaN,NaN,NaN,16.7,NaN,NaN,NaN,NaN,1.6,3.3,4.9,0.112,NaN,NaN,NaN,NaN,413.0,897.0,0.460,NaN,NaN,NaN,413.0,897.0,0.460,0.460,145.0,176.0,0.824,NaN,NaN,239.0,268.0,NaN,NaN,NaN,219.0,971.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,NaN,0.00,-0.432,-8.64,1.0,134.0,3.0,2
2895,1970.0,Bill Bradley*,SF,26.0,NYK,67.0,NaN,2098.0,13.7,0.498,NaN,0.196,NaN,NaN,NaN,16.7,NaN,NaN,NaN,NaN,1.6,3.3,4.9,0.112,NaN,NaN,NaN,NaN,413.0,897.0,0.460,NaN,NaN,NaN,413.0,897.0,0.460,0.460,145.0,176.0,0.824,NaN,NaN,239.0,268.0,NaN,NaN,NaN,219.0,971.0,Bill Bradley,1968,1977,F-G,6-5,205.0,1943-07-28,Princeton University,NaN,0.00,-0.432,-8.64,1.0,134.0,1.0,2
3107,1971.0,Bill Bradley*,SF,27.0,NYK,78.0,NaN,2300.0,13.3,0.490,NaN,0.192,NaN,NaN,5.2,16.5,NaN,NaN,NaN,NaN,1.8,3.6,5.4,0.114,NaN,NaN,NaN,NaN,413.0,912.0,0.453,NaN,NaN,NaN,413.0,912.0,0.453,0.453,144.0,175.0,0.823,NaN,NaN,260.0,280.0,NaN,NaN,NaN,245.0,970.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,NaN,0.00,-0.432,-8.64,1.0,156.0,3.0,2
3108,1971.0,Bill Bradley*,SF,27.0,NYK,78.0,NaN,2300.0,13.3,0.490,NaN,0.192,NaN,NaN,5.2,16.5,NaN,NaN,NaN,NaN,1.8,3.6,5.4,0.114,NaN,NaN,NaN,NaN,413.0,912.0,0.453,NaN,NaN,NaN,413.0,912.0,0.453,0.453,144.0,175.0,0.823,NaN,NaN,260.0,280.0,NaN,NaN,NaN,245.0,970.0,Bill Bradley,1968,1977,F-G,6-5,205.0,1943-07-28,Princeton University,NaN,0.00,-0.432,-8.64,1.0,156.0,1.0,2
3353,1972.0,Bill Bradley*,SF,28.0,NYK,78.0,NaN,2780.0,12.9,0.502,NaN,0.183,NaN,NaN,4.4,16.0,NaN,NaN,NaN,NaN,2.7,2.8,5.4,0.094,NaN,NaN,NaN,NaN,504.0,1085.0,0.465,NaN,NaN,NaN,504.0,1085.0,0.465,0.465,169.0,199.0,0.849,NaN,NaN,250.0,315.0,NaN,NaN,NaN,254.0,1177.0,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN,NaN,0.00,-0.432,-8.64,1.0,156.0,3.0,2
3354,1972.0,Bill Bradley*,SF,28.0,NYK,78.0,NaN,2780.0,12.9,0.502,NaN,0.183,NaN,NaN,4.4,16.0,NaN,NaN,NaN,NaN,2.7,2.8,5.4,0.094,NaN,NaN,NaN,NaN,504.0,1085.0,0.465,NaN,NaN,NaN,504.0,1085.0,0.465,0.465,169.0,199.0,0.849,NaN,NaN,250.0,315.0,NaN,NaN,NaN,254.0,1177.0,Bill Bradley,1968,1977,F-G,6-5,205.0,1943-07-28,Princeton University,NaN,0.00,-0.432,-8.64,1.0,156.0,1.0,2
